# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    print("root",root)
    print()
    print("dirs",dirs)
    print()
    print("files",files)
    print()

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print("file_path_list",file_path_list)
print()

/home/workspace
root /home/workspace/event_data

dirs []

files ['2018-11-14-events.csv', '2018-11-26-events.csv', '2018-11-22-events.csv', '2018-11-04-events.csv', '2018-11-15-events.csv', '2018-11-05-events.csv', '2018-11-18-events.csv', '2018-11-13-events.csv', '2018-11-19-events.csv', '2018-11-17-events.csv', '2018-11-09-events.csv', '2018-11-08-events.csv', '2018-11-30-events.csv', '2018-11-25-events.csv', '2018-11-20-events.csv', '2018-11-01-events.csv', '2018-11-10-events.csv', '2018-11-07-events.csv', '2018-11-12-events.csv', '2018-11-06-events.csv', '2018-11-29-events.csv', '2018-11-23-events.csv', '2018-11-02-events.csv', '2018-11-27-events.csv', '2018-11-03-events.csv', '2018-11-24-events.csv', '2018-11-21-events.csv', '2018-11-11-events.csv', '2018-11-16-events.csv', '2018-11-28-events.csv']

file_path_list ['/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/works

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [21]:
# make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [22]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [23]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [24]:
##Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(sessionid int, iteminsession int, artist text, song_title text,length float, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                

##### As we know we are going to find data based on sessionid and iteminsesssion, we will set this two columns as our primary key and partitioned by sessionid

##### as required, we will also select artist, song_title,length to validafy the data

In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (artist, song_title, length, sessionid, iteminsession)"
        query = "INSERT INTO song_library (sessionid, iteminsession, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ##Assign column element in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [26]:
##verify the data was entered into the table

query = "select artist, song_title, length from song_library WHERE sessionid=338 AND iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


##### Summary: Here we can see that the Artist's name is Faithless, Song's name is Music Matters (Mark Knight Dub) and the song's Length is 495.30731201171875

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [27]:
##Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_info "
query = query + "(sessionid int, userid int, iteminsession int, artist text, song_title text, firstname text, lastname text, PRIMARY KEY ((sessionid, userid),iteminsession));"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_info (sessionid, userid, iteminsession, artist, song_title, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

##### As we know we are going to find data based on userid and sessionid, and it also required to be sort by iteminsession. we will set this three columns as our primary key and partitioned by userid and sessionid

##### as required, we will also select artist, song_title, firstname, lastname to validafy the data

In [32]:
query = "select artist, song_title, firstname, lastname from user_info WHERE sessionid=182 AND userid=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


##### Summary: Here we can see that there are four results were selected. And they were sorted by iteminsession.
##### the artists are Down To The Bone, Three Drives, Sebastien Tellier, Lonnie Gordon
##### the songs are Keep On Keepin' On, Greece 2000, Kilometer, Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
##### the user is Sylvie Cruz

In [13]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS music_log "
query = query + "(song_title text, userid int, firstname text, lastname text, PRIMARY KEY (song_title, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_log (song_title, userid, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

##### As we know we are going to find data based on song_title, but it's not unique enough. so we will set song_title and userid as our primary key and partitioned by userid

##### as required, we will also select firstname and lastname to validafy the data

In [14]:
query = "select firstname,lastname from music_log WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


##### Summary: Here we can see that there are three results were selected. 
##### the users are Jacqueline Lynch, Tegan Levine, Sara Johnson

### Drop the tables before closing out the sessions

In [15]:
##Drop the table before closing out the sessions

query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_info"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_log"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()